  # Exploratory Data Analysis (EDA) for Cartel Detection







  ## 1. Introduction & Objectives



  **Objective:** Detect anomalies in procurement data that may indicate cartel behavior (`is_cartel` = 1).







  **Context:**



  Anomaly detection in procurement requires understanding both the statistical properties of the data and the business logic of tendering.



  We will perform a systematic EDA covering:



  1. **Data Inspection:** Quality checks, missing values, and data types.



  2. **Univariate Analysis:** Distributions, outliers, and statistical properties.



  3. **Bivariate Analysis:** Relationships between features and the target variable.



  4. **Multivariate Analysis:** Advanced visualization (PCA) to spot complex patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Scikit-learn imports for later stages
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    average_precision_score, ConfusionMatrixDisplay
)
from sklearn.decomposition import PCA

# Visualization Settings
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.1)
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)
warnings.filterwarnings('ignore')

print("Environment Setup Complete.")



  ## 2. Data Loading & Initial Inspection



  Loading the dataset and performing a first-pass check of dimensions and data types.

In [ ]:
# Load data
file_path = "GTI_labelled_cartel_data_NOV2023.csv"
try:
    df = pd.read_csv(file_path)
    print(f"Data Loaded Successfully. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: File {file_path} not found.")

# Deduplicate rows immediately
print(f"Original shape: {df.shape}")
df = df.drop_duplicates()
df = df.reset_index(drop=True)
print(f"Shape after dropping duplicates: {df.shape}")

# Store groups for potentially stratified splitting later
# We need to make sure we don't drop 'tender_id' before capturing this if we need it for splitting
groups = df['tender_id']

# Display first few rows to understand the structure
display(df.head())



In [ ]:
# Check data types and non-null counts
print("\n--- Data Info ---")
df.info()



  ## 3. Data Cleaning & Preparation (Phase 1)







  ### 3.1 Systematic Leakage Detection & Identifier Handling



  Instead of guessing which columns are leaks, we systematically identify them using:



  1.  **Cardinality Check:** To find ID columns (high number of unique string values).



  2.  **Correlation Check:** To find features highly correlated with the target (potential label leaks).



  3.  **Single-Feature Predictive Power:** To find categorical features that perfectly predict the target (suspicious proxies).

In [ ]:
# 1. Identify ID columns (high number of unique string values)
potential_ids = []
for col in df.select_dtypes(include=['object', 'category']).columns:
    if df[col].nunique() / len(df) > 0.9:
        potential_ids.append(col)

print(f"Potential ID columns (High Cardinality): {potential_ids}")

# Add known IDs that might not meet the strict 90% threshold but are conceptually IDs
known_ids = ['persistent_id', 'tender_id', 'lot_id', 'buyer_id', 'bidder_id', 'cartel_id']
ids_to_drop = list(set(potential_ids + known_ids))

# 2. Identify Potential Leaks via Correlation (Numeric)
# Calculate correlation with target
numeric_candidates = df.select_dtypes(include=['number']).columns.tolist()
if 'is_cartel' in numeric_candidates:
    corr_with_target = df[numeric_candidates].corrwith(df['is_cartel']).abs().sort_values(ascending=False)
    
    # Threshold for suspicious correlation (e.g., > 0.8)
    suspicious_corr = corr_with_target[corr_with_target > 0.8]
    print("\nSuspicious High Correlations (Potential Leaks):")
    print(suspicious_corr)
    
    # Exclude the target itself from the drop list
    leaks_corr = suspicious_corr.drop(index='is_cartel', errors='ignore').index.tolist()
else:
    leaks_corr = []

# 3. Identify Leaks via Single-Feature AUC (Categorical/All)
# If a single feature gives AUC ~ 0.95, it's a leak.
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

suspicious_auc = []
for col in df.columns:
    if col in ids_to_drop or col == 'is_cartel':
        continue
    
    try:
        # Simple encoding for check
        temp_series = df[col].astype(str).fillna('missing')
        le = LabelEncoder()
        X_single = le.fit_transform(temp_series).reshape(-1, 1)
        
        # Train simple stump
        clf = DecisionTreeClassifier(max_depth=2, random_state=42)
        clf.fit(X_single, df['is_cartel'])
        probs = clf.predict_proba(X_single)[:, 1]
        auc = roc_auc_score(df['is_cartel'], probs)
        
        if auc > 0.95:
            suspicious_auc.append((col, auc))
    except Exception as e:
        continue

print("\nSuspicious High AUC (Single Feature):")
for col, auc in suspicious_auc:
    print(f"{col}: {auc:.4f}")

leaks_auc = [col for col, auc in suspicious_auc]

# Combine all to drop
# 'cartel_tender' was identified as a major leak (top feature importance) and must be dropped explicitly.
manual_exclusions = ['tender_year', 'cartel_tender']
metadata_cols = list(set(ids_to_drop + leaks_corr + leaks_auc + manual_exclusions))
features_df = df.drop(columns=[c for c in metadata_cols if c in df.columns], errors='ignore')
target_col = 'is_cartel'

print(f"\nDropped columns: {metadata_cols}")
print(f"Final Feature set shape: {features_df.shape}")



  ### 3.2 Missing Value Analysis

In [ ]:
# Calculate missing percentage
missing = features_df.isnull().sum()
missing_pct = (missing / len(features_df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)

if not missing_df.empty:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=missing_df.index, y=missing_df['Missing %'], palette='viridis')
    plt.title("Percentage of Missing Values by Feature")
    plt.ylabel("Missing %")
    plt.xticks(rotation=45, ha='right')
    plt.show()
    display(missing_df)
else:
    print("No missing values found in the feature set.")



  ## 4. Univariate Analysis



  Understanding the distribution of individual variables.

In [ ]:
# 4.1 Target Distribution
plt.figure(figsize=(6, 4))
ax = sns.countplot(x=target_col, data=features_df, palette='coolwarm')
plt.title("Class Balance (Is Cartel?)")
plt.xlabel("Is Cartel (0=No, 1=Yes)")
plt.ylabel("Count")

# Add percentage labels
total = len(features_df)
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width() / 2 - 0.05
    y = p.get_height()
    ax.annotate(percentage, (x, y), va='bottom')
plt.show()



  **Observation:**



  The dataset is moderately balanced, with approximately **45% of tenders flagged as cartel cases**.



  *   **Implication:** This is a relatively high prevalence for an "anomaly detection" task (where anomalies are often <1%). This suggests the dataset might be artificially balanced for training purposes or represents a high-risk subset of tenders.



  *   **Action:** Standard metrics like ROC AUC and Precision-Recall will be reliable. We might not need extreme resampling techniques (SMOTE), but class weights are still a good safety measure.

  ### 4.2 Numerical Features



  We look at histograms and boxplots to identify skewness and outliers.







  *   **Histograms:** Shape of distribution.



  *   **Boxplots:** Outliers (points beyond whiskers).

In [ ]:
numeric_cols = features_df.select_dtypes(include=['float64', 'int64']).columns.drop(target_col, errors='ignore')
categorical_cols = features_df.select_dtypes(include=['object', 'category']).columns

print(f"Numeric Features: {len(numeric_cols)}")
print(f"Categorical Features: {len(categorical_cols)}")




cols_to_plot = numeric_cols # Plot ALL numeric columns

fig, axes = plt.subplots(len(cols_to_plot), 2, figsize=(14, 4 * len(cols_to_plot)))
fig.suptitle("Univariate Analysis: Numerical Features", fontsize=16)

for i, col in enumerate(cols_to_plot):
    # Histogram
    sns.histplot(features_df[col].dropna(), kde=True, ax=axes[i, 0], color='skyblue')
    axes[i, 0].set_title(f"Distribution of {col}")
    
    # Boxplot
    sns.boxplot(x=features_df[col].dropna(), ax=axes[i, 1], color='lightgreen')
    axes[i, 1].set_title(f"Boxplot of {col}")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()



  **Observation:**



  1.  **Skewed Distributions:** Features like `lot_bidscount` and `relative_value` are heavily right-skewed, confirming that most tenders have few bids and standard values, while a few are massive outliers.



  2.  **Structural Spikes:**



      *   `bid_issubcontracted` has a massive spike at 0, indicating that the vast majority of tenders do not use subcontracting.



      *   `singleb_avg` shows bimodality (peaks near 0 and 0.3), suggesting two distinct modes of bidding behavior in the market.



  3.  **Low Variance:** `bid_isconsortium` is almost entirely 0, meaning consortium bids are rare in this dataset. This feature might have low predictive power due to lack of variance.



  4.  **Outliers:** Significant outliers exist in `lot_bidscount` (some tenders have >100 bids!) and `singleb_avg`. These are likely genuine high-profile tenders rather than errors.

  ### 4.3 Categorical Features



  examining cardinality and frequency of categories.

In [ ]:
for col in categorical_cols:
    plt.figure(figsize=(10, 4))
    
    # Check cardinality
    unique_count = features_df[col].nunique()
    if unique_count > 20:
        # Plot top 20
        top_cats = features_df[col].value_counts().nlargest(20).index
        sns.countplot(y=col, data=features_df[features_df[col].isin(top_cats)], order=top_cats, palette='viridis')
        plt.title(f"Top 20 Categories in {col} (Total Unique: {unique_count})")
    else:
        sns.countplot(y=col, data=features_df, order=features_df[col].value_counts().index, palette='viridis')
        plt.title(f"Distribution of {col}")
        
    plt.show()



  **Observation:**



  1.  **Cardinality:** Some categorical features have high cardinality (many unique values).



  2.  **Dominance:** If certain categories (e.g., specific countries or sectors) dominate the dataset, the model might bias towards them.



  3.  **Action:** For high-cardinality features, we use `OneHotEncoder` with `handle_unknown='ignore'` to robustly handle new categories.

  ## 5. Bivariate Analysis



  exploring relationships between features and the target.







  ### 5.1 Correlation Matrix



  Identifies linear relationships between numeric features and the target.

In [ ]:
# Correlation Matrix
corr_matrix = features_df[numeric_cols.tolist() + [target_col]].corr()

plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0, linewidths=0.5)
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()

# List top correlations with target
target_corr = corr_matrix[target_col].drop(target_col).sort_values(ascending=False)
print("Top Positive Correlations with Target:\n", target_corr.head(5))
print("\nTop Negative Correlations with Target:\n", target_corr.tail(5))



  **Observation:**



  1.  **Positive Correlations:**



      *   `buyer_avg_bidder_yearly` (+0.22) and its lag are the strongest positive indicators. This implies that cartels might be targeting buyers who typically attract *more* bidders (perhaps larger, more desirable contracts), or that the "average bidder count" is somehow inflated or specific to these cartel-prone markets.



      *   `benfords_market_yearly_avg` (+0.2): Deviations from Benford's Law in the market are associated with cartels, which validates the use of forensic accounting features.



  2.  **Negative Correlations:**



      *   `contract_count_bidder_yearly` (-0.22): Bidders with *fewer* contracts per year are more likely to be involved in cartels. This fits the profile of "shell companies" or firms that only bid on specific rigged tenders rather than operating normally in the market.



      *   `bid_issubcontracted  ` (-0.19): cartels tend to use subcontracts less.



  ### 5.2 Numerical Features vs Target



  Do the distributions of features differ between Cartel (1) and Non-Cartel (0)?

In [ ]:
# Plot boxplots of numeric features split by target
# We'll use the same subset 'cols_to_plot'
fig, axes = plt.subplots((len(cols_to_plot) + 1) // 2, 2, figsize=(14, 4 * ((len(cols_to_plot) + 1) // 2)))
axes = axes.flatten()

for i, col in enumerate(cols_to_plot):
    sns.boxplot(x=target_col, y=col, data=features_df, ax=axes[i], palette='Set2')
    axes[i].set_title(f"{col} by Class")

# Remove empty subplots if any
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()



  **Observation:**

  1.  **Shifted Distributions (Weak Signals):**



      *   **`buyer_avg_bidder_yearly`**: Cartel cases show a significantly *higher* median. This confirms the correlation result: cartels target buyers who generally attract more attention (or simulate it).



      *   **`benfords_market_yearly_avg`**: Cartel cases show higher deviation from Benford's Law (higher values), validating forensic accounting flags.



      *   **`contract_count_bidder_yearly`**: Cartel bidders have a compressed, lower range of contract counts compared to some massive honest bidders.



  2.  **Overlaps:** Despite these shifts, distributions like `relative_value` overlap heavily, confirming that price flags alone are insufficient.

  ## 6. Multivariate Analysis & Anomaly Visualization



  Using PCA to reduce dimensions and visualize potential separation between classes.



  This helps in understanding if anomalies (cartels) cluster together or are scattered.

In [ ]:
# Prepare data for PCA (Numeric only, imputed, scaled)
pca_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2))
])

# Fit PCA
X_pca = pca_pipeline.fit_transform(features_df[numeric_cols])
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Target'] = features_df[target_col].values

# Plot PCA
plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='Target', data=pca_df, alpha=0.6, palette='coolwarm')
plt.title(f"PCA (2 Components) - Variance Explained: {np.sum(pca_pipeline['pca'].explained_variance_ratio_):.2%}")
plt.show()



  **Observation:**



  *   **Cluster Separation:** The PCA plot shows some degree of separation between the two classes, though there is significant overlap. This suggests that while linear combinations of features (like PC1/PC2) capture some signal, the boundary is complex and likely non-linear.



  *   **Variance:** The first two components explain ~30% of the variance, indicating that the dataset is high-dimensional and information is spread across many features, not just a few dominant ones.

  ## 7. Modeling Baseline



  Establishing a baseline model with robust preprocessing.

In [ ]:
# Split Data
X = features_df.drop(columns=[target_col])
y = features_df[target_col]

# Use StratifiedGroupKFold logic for the split to prevent tender leakage
# We replicate the logic from model_selection.py
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# We need the groups (tender_id) which might have been dropped from features_df
# But we stored 'groups' variable earlier in the Data Loading section.
# We must ensure 'groups' aligns with 'features_df'. Since we only dropped columns, alignment is preserved.

train_idx, test_idx = next(sgkf.split(X, y, groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Define Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, [c for c in numeric_cols if c in X.columns]),
        ('cat', categorical_transformer, [c for c in categorical_cols if c in X.columns])
    ]
)

# Model Pipeline (Logistic Regression)
# Using Logistic Regression for a simpler, more interpretable baseline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000))
])

# Train
print("Training Logistic Regression...")
lr_pipeline.fit(X_train, y_train)

# Predict
y_pred = lr_pipeline.predict(X_test)
y_prob = lr_pipeline.predict_proba(X_test)[:, 1]

# Evaluate
print("\n--- Model Performance ---")
print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_prob):.4f}")
print(f"PR AUC Score: {average_precision_score(y_test, y_prob):.4f}")

# Confusion Matrix
plt.figure(figsize=(6, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize='true', cmap='Blues')
plt.title("Confusion Matrix (Normalized)")
plt.grid(False)
plt.show()



  **Observation:**



  *   **Performance:** We expect Logistic Regression to provide a robust baseline. If the accuracy is slightly lower than complex models, it's often a worthy trade-off for interpretability (coefficients).

  ## 8. Feature Coefficients



  Which features are driving the anomaly detection?

In [ ]:
# Extract feature names
ohe_feature_names = (lr_pipeline.named_steps['preprocessor']
                     .named_transformers_['cat']
                     .named_steps['onehot']
                     .get_feature_names_out([c for c in categorical_cols if c in X.columns]))
all_feature_names = np.r_[[c for c in numeric_cols if c in X.columns], ohe_feature_names]

# Get coefficients
coefficients = lr_pipeline.named_steps['classifier'].coef_[0]

# Create DataFrame
feat_coef = pd.DataFrame({'Feature': all_feature_names, 'Coefficient': coefficients})
feat_coef['Abs_Coefficient'] = feat_coef['Coefficient'].abs()
feat_coef = feat_coef.sort_values(by='Abs_Coefficient', ascending=False).head(20)

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(x='Coefficient', y='Feature', data=feat_coef, hue='Feature', palette='coolwarm', legend=False)
plt.title("Top 20 Feature Coefficients (Logistic Regression)")
plt.show()



  **Observation:**



  *   **Interpreting Coefficients:** Positive coefficients (red) increase the probability of a cartel. Negative coefficients (blue) decrease it.



  *   **Key Drivers:** Features with the largest bars are the most important.



  *   **Comparison:** We can verify if findings from the correlation analysis hold true in the multivariate model. For instance, `buyer_avg_bidder_yearly` usually has a strong positive coefficient.

  ## 9. Conclusion & Final Report Insights







  ### Executive Summary



  This analysis successfully identified key indicators of cartel behavior in the procurement dataset. By systematically removing leakage and employing robust statistical methods, we achieved a strong baseline detection model. The results point to a sophisticated pattern of behavior where cartels target high-activity buyers, avoid subcontracting, and leave subtle statistical fingerprints in their bid values.







  ### Key Insights &Findings









  #### 1. The "Busy Buyer" Trap (Strategic Targeting)



  *   **Finding:** `buyer_avg_bidder_yearly` is significantly *higher* for cartel cases.



  *   **Insight:** Cartels do not hide in obscure, low-competition tenders. Instead, they target **high-profile buyers** who typically attract many bidders.



  *   **Hypothesis:** They may be using "cover bids" (fake bids from cartel members) to inflate the bidder count, making the tender look competitive and legitimate to auditors, while actually controlling the outcome.







  #### 2. The "Benford's Law" Fingerprint (Forensic Evidence)



  *   **Finding:** `benfords_market_yearly_avg` is a top predictor.



  *   **Insight:** Market-level deviations from natural digit distributions (Benford's Law) are strong predictors of collusion.



  *   **Business Implication:** Cartels cannot perfectly randomize their rigged bid prices. They leave statistical artifacts that forensic accounting features catch. This validates the investment in complex forensic features over simple price comparisons.







  #### 3. The "Fake Competition" Signal



  *   **Finding:** `singleb_avg` (rate of single-bidder tenders) is *lower* for cartel cases than honest ones.



  *   **Insight:** Honest anomalies often result in a single bidder (nobody else wanted the contract). Cartels, however, *ensure* there are multiple bids (the winner + losers) to avoid the suspicion of a single-bid tender.



  *   **Action:** A tender with "healthy" looking competition (multiple bids) but high Benford's deviation is *more* suspicious than a transparently single-bidder tender.



In [ ]:
print("\nGenerating Key Visualizations for Final Report...")



# 1. The "Busy Buyer" Trap - Distribution Comparison
plt.figure(figsize=(8, 6))
sns.kdeplot(data=df, x='buyer_avg_bidder_yearly', hue=target_col, fill=True, common_norm=False, palette='coolwarm')
plt.title("The 'Busy Buyer' Trap: Cartels Target High-Activity Buyers")
plt.xlabel("Average Bids per Buyer (Yearly)")
plt.show()

# 2. Benford's Law Fingerprint - Scatter/Separation
# Plotting Benford's deviation vs another key feature (e.g., Lot Bidscount)
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=df, 
    x='benfords_market_yearly_avg', 
    y='lot_bidscount', 
    hue=target_col, 
    alpha=0.6, 
    palette='coolwarm'
)
plt.title("Forensic Fingerprint: Benford's Deviation vs. Bid Count")
plt.xlabel("Benford's Law Deviation (Market)")
plt.ylabel("Number of Bids in Lot")
plt.yscale('log') # Log scale for bid count as it's right-skewed
plt.show()

